# Урок 13. Модель BERT и GPT   
Задание 13
Взять датасет https://huggingface.co/datasets/merionum/ru_paraphraser решить задачу парафраза

In [ ]:
from transformers import *

In [2]:
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
import json
import numpy as np
import pandas as pd

In [4]:
from tqdm.notebook import tqdm

In [5]:
data_train = pd.read_json("../data/train_paraphr.jsonl", lines=True)

In [6]:
data_test = pd.read_json("../data/test_paraphr.jsonl", lines=True)

**as there was no model provided, we'll choose smth from https://huggingface.co/docs/transformers**

In [7]:
model = PegasusForConditionalGeneration.from_pretrained("tuner007/pegasus_paraphrase")


loading configuration file config.json from cache at C:\Users\Armik/.cache\huggingface\hub\models--tuner007--pegasus_paraphrase\snapshots\0159e2949ca73657a2f1329898f51b7bb53b9ab2\config.json
Model config PegasusConfig {
  "activation_dropout": 0.1,
  "activation_function": "relu",
  "add_bias_logits": false,
  "add_final_layer_norm": true,
  "architectures": [
    "PegasusForConditionalGeneration"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 16,
  "decoder_start_token_id": 0,
  "dropout": 0.1,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 16,
  "eos_token_id": 1,
  "extra_pos_embeddings": 1,
  "force_bos_token_to_be_generated": false,
  "forced_eos_token_id": 1,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is

In [8]:
tokenizer = PegasusTokenizerFast.from_pretrained("tuner007/pegasus_paraphrase")

loading file spiece.model from cache at C:\Users\Armik/.cache\huggingface\hub\models--tuner007--pegasus_paraphrase\snapshots\0159e2949ca73657a2f1329898f51b7bb53b9ab2\spiece.model
loading file tokenizer.json from cache at None
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at C:\Users\Armik/.cache\huggingface\hub\models--tuner007--pegasus_paraphrase\snapshots\0159e2949ca73657a2f1329898f51b7bb53b9ab2\special_tokens_map.json
loading file tokenizer_config.json from cache at C:\Users\Armik/.cache\huggingface\hub\models--tuner007--pegasus_paraphrase\snapshots\0159e2949ca73657a2f1329898f51b7bb53b9ab2\tokenizer_config.json
loading configuration file config.json from cache at C:\Users\Armik/.cache\huggingface\hub\models--tuner007--pegasus_paraphrase\snapshots\0159e2949ca73657a2f1329898f51b7bb53b9ab2\config.json
Model config PegasusConfig {
  "_name_or_path": "tuner007/pegasus_paraphrase",
  "activation_dropout": 0.1,
  "activation_function": "r

In [9]:
def get_paraphrased_sentences(model, tokenizer, sentence, num_return_sequences=1, num_beams=1):
  # tokenize the text to be form of a list of token IDs
    inputs = tokenizer([sentence], truncation=True, padding="longest", return_tensors="pt")
  # generate the paraphrased sentences
    outputs = model.generate(
    **inputs,
        num_beams=num_beams,
        num_return_sequences=num_return_sequences,
  )
  # decode the generated sentences using the tokenizer to get them back to text
    return tokenizer.batch_decode(outputs, skip_special_tokens=True)

In [10]:
data_train.head()

,id,id_1,id_2,text_1,text_2,class
0,1,201,8159,Полицейским разрешат стрелять на поражение по ...,Полиции могут разрешить стрелять по хулиганам ...,0
1,2,202,8158,Право полицейских на проникновение в жилище ре...,Правила внесудебного проникновения полицейских...,0
2,3,273,8167,Президент Египта ввел чрезвычайное положение в...,Власти Египта угрожают ввести в стране чрезвыч...,0
3,4,220,8160,Вернувшихся из Сирии россиян волнует вопрос тр...,Самолеты МЧС вывезут россиян из разрушенной Си...,-1
4,5,223,8160,В Москву из Сирии вернулись 2 самолета МЧС с р...,Самолеты МЧС вывезут россиян из разрушенной Си...,0


In [11]:
get_paraphrased_sentences(model, tokenizer,
                          "Alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do: once or twice she had peeped into the book her sister was reading, but it had no pictures or conversations in it, 'and what is the use of a book,' thought Alice 'without pictures or conversation?")

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 0,
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "forced_eos_token_id": 1,
  "length_penalty": 0.8,
  "max_length": 60,
  "num_beams": 8,
  "pad_token_id": 0,
  "transformers_version": "4.29.2"
}

C:\Users\Armik\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\generation\utils.py:1346: UserWarning: Using `max_length`'s default (60) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


['Alice was tired of sitting by her sister on the bank and of having nothing to do, and she had peeped into the book her sister was reading, but it had no pictures or conversations in it.']

Alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do: once or twice she had peeped into the book her sister was reading, but it had no pictures or conversations in it, 'and what is the use of a book,' thought Alice 'without pictures or conversation?'

In [12]:
get_paraphrased_sentences(model, tokenizer,
                          'In another moment down went Alice after it, never once considering how in the world she was to get out again.')

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 0,
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "forced_eos_token_id": 1,
  "length_penalty": 0.8,
  "max_length": 60,
  "num_beams": 8,
  "pad_token_id": 0,
  "transformers_version": "4.29.2"
}



['Alice never once considered how she would get out again, after she had just gone down.']

Original:    
In another moment down went Alice after it, never once considering how in the world she was to get out again.

**it's really impressive, but it's not a multilingual model, so for russian we have to look for smth else**

**source**   
*https://huggingface.co/datasets/inkoziev/paraphrases*   
*https://github.com/Koziev/paraphraser/tree/main*

In [13]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM


In [14]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model_name = "inkoziev/paraphraser"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
model.to(device)
model.eval()


loading file vocab.json from cache at C:\Users\Armik/.cache\huggingface\hub\models--inkoziev--paraphraser\snapshots\596b12ed77fd66f3216fcd6021d8ae361db7462c\vocab.json
loading file merges.txt from cache at C:\Users\Armik/.cache\huggingface\hub\models--inkoziev--paraphraser\snapshots\596b12ed77fd66f3216fcd6021d8ae361db7462c\merges.txt
loading file tokenizer.json from cache at C:\Users\Armik/.cache\huggingface\hub\models--inkoziev--paraphraser\snapshots\596b12ed77fd66f3216fcd6021d8ae361db7462c\tokenizer.json
loading file added_tokens.json from cache at C:\Users\Armik/.cache\huggingface\hub\models--inkoziev--paraphraser\snapshots\596b12ed77fd66f3216fcd6021d8ae361db7462c\added_tokens.json
loading file special_tokens_map.json from cache at C:\Users\Armik/.cache\huggingface\hub\models--inkoziev--paraphraser\snapshots\596b12ed77fd66f3216fcd6021d8ae361db7462c\special_tokens_map.json
loading file tokenizer_config.json from cache at C:\Users\Armik/.cache\huggingface\hub\models--inkoziev--paraphr

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50259, 1536)
    (wpe): Embedding(2048, 1536)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-23): 24 x GPT2Block(
        (ln_1): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1536, out_features=50259, bias=False)
)

In [15]:
def get_paraphr_rus(seed, num_return_sequences=10):
   
    encoded_prompt = tokenizer.encode("<s>" + seed + "<sep>", add_special_tokens=False, return_tensors="pt").to(device)
    output_sequences = model.generate(input_ids=encoded_prompt,
                                      max_length=100,
                                      typical_p=0.85,
                                      top_k=0,
                                      top_p=1.0,
                                      do_sample=True,
                                      num_return_sequences=num_return_sequences,
                                      pad_token_id=tokenizer.pad_token_id)
    all_texts = []
    for o in output_sequences:
        text = tokenizer.decode(o.tolist(), clean_up_tokenization_spaces=True)
        text = text[text.index('<sep>') + 5:]
        text = text[: text.find('</s>')]
  #      print(text)
        all_texts.append(text)
        
    return all_texts

In [16]:
paraphr_0 = get_paraphr_rus(data_train['text_1'][0])
paraphr_0

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.29.2"
}



['Полиция будет использовать оружие для пресечения беспорядков',
 'Полиция не будет возражать, если граждане получат ранения в случае стрельбы на поражение.',
 'Оперативные службы получат право на расстрел граждан, у которых диагностирована травматическая болезнь',
 'Полиция будет применять оружие, если это необходимо, для защиты граждан.',
 'В полиции разрешат использовать оружие для защиты граждан.',
 'МВД подготовило поправки в статью о применении силы в отношении граждан с травмами.',
 'Правоохранительные органы получат право применять оружие по отношению к гражданам, получившим ранения при задержании, - таков следующий шаг в рамках закона об ужесточении ответственности за побо резкого сопротивления полиции',
 'В МВД рассматривают возможность разрешения полицейским стрелять на поражение по гражданам с травмами.',
 'полиция планирует разрешить использовать для стрельбы на поражение граждан без травматики',
 'Полицейским дадут право стрелять на поражение граждан, если есть травма.']

In [17]:
data_train['text_1'][0]

'Полицейским разрешат стрелять на поражение по гражданам с травматикой.'

**some of them are really funny but it works in some cases even with an unconventional word "травматика"**

In [18]:
paraphr_27 = get_paraphr_rus(data_train['text_1'][27])
paraphr_27

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.29.2"
}



['Число погибших в Сирии достигло 60 тысяч',
 'Пятнадцать тысяч человек погибли в Сирии в ходе конфликта',
 'Число погибших при подавлении антиправительственных выступлений в Сирии достигло 61 тысячи',
 'В результате столкновений в Сирии погибли десятки тысяч человек',
 'Число погибших при столкновениях в Сирии возросло до 60 тысяч',
 'Число жертв конфликта в Сирии возросло до 60 тысяч',
 'В ходе вооруженного конфликта в Сирии погибли более 60 тысяч человек.',
 'Число жертв конфликта в Сирии достигло рекордной отметки в 60 тысяч',
 'Число жертв теракта в Сирии достигло 60 тысяч',
 'Число погибших в ходе беспорядков в Сирии достигло более 60 тысяч']

In [19]:
data_train['text_1'][27]

'Число убитых в ходе конфликта в Сирии превысило 60 тысяч.'

In [20]:
data_train_true = data_train.loc[data_train['class']==1, ['text_1', 'text_2']]

In [21]:
data_train_true['paraphr'] = ''

In [22]:
for row_num in range(len(data_train_true['text_1'])):
    data_train_true['paraphr'].iloc[row_num] = get_paraphr_rus(data_train_true['text_1'].iloc[row_num],
                                                               num_return_sequences=1)[0];
    

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.29.2"
}

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.29.2"
}

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.29.2"
}

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.29.2"
}

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.29.2"
}

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.29.2"
}

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id":

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.29.2"
}

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.29.2"
}

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.29.2"
}

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.29.2"
}

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.29.2"
}

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.29.2"
}

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id":

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.29.2"
}

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.29.2"
}

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.29.2"
}

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.29.2"
}

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.29.2"
}

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.29.2"
}

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id":

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.29.2"
}

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.29.2"
}

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.29.2"
}

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.29.2"
}

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.29.2"
}

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.29.2"
}

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id":

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.29.2"
}

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.29.2"
}

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.29.2"
}

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.29.2"
}

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.29.2"
}

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.29.2"
}

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id":

...

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.29.2"
}

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.29.2"
}

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.29.2"
}

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.29.2"
}

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.29.2"
}

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.29.2"
}

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id":

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.29.2"
}

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.29.2"
}

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.29.2"
}

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.29.2"
}

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.29.2"
}

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.29.2"
}

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id":

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.29.2"
}

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.29.2"
}

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.29.2"
}

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.29.2"
}

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.29.2"
}

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.29.2"
}

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id":

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.29.2"
}

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.29.2"
}

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.29.2"
}

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.29.2"
}

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.29.2"
}

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.29.2"
}

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id":

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.29.2"
}

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.29.2"
}

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.29.2"
}

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.29.2"
}

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.29.2"
}

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.29.2"
}

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id":

In [23]:
data_train_true

,text_1,text_2,paraphr
5,Приставы соберут отпечатки пальцев российских ...,Приставы снимут отпечатки пальцев у злостных н...,В России приставы получат отпечатки пальцев до...
10,Москвичи смогут забронировать в Интернете мест...,В Москве можно будет забронировать место на кл...,Могилу могут забронировать в Интернете.
13,Северокорейский лидер впервые за 19 лет поздра...,Лидер КНДР впервые за 19 лет поздравил согражд...,Премьер-министр КНДР впервые продукции фабрики...
14,Мужчина из Подмосковья случайно убил жену в Но...,Житель Подмосковья случайно убил жену на новог...,Случайное убийство в подмосковном Снегири удал...
17,Житель Украины расстрелял посетителей кафе.,На Украине мужчина через окно расстрелял посет...,В Киеве был избит посетитель кафе за отказ пла...
...,...,...,...
7182,"Кадыров подарил ""ночным волкам"" новые мотоциклы",Кадыров подарил мотоциклы чеченским «Ночным во...,Ночным волкам подарили мотоциклы Кадырова
7183,Полиция в США опубликовала видео избиения сотр...,В США опубликовали видео избиения полицейским ...,Полиция США опубликовала кадры избиения чернок...
7185,Следственный комитет России заявил о несанкцио...,Следственный комитет сообщил о взломе своего с...,Следственный комитет России заявил о взломе са...
7188,Суд оправдал Васильеву в хищении акций на два ...,Суд снял с Васильевой обвинения в хищении 2 мл...,В деле Васильевой появился новый эпизод мошенн...


**not bad. processing took eight hours, so we'll save the result to a file to be able to evaluate it later if needed**

In [29]:
with open('../data/df_paraphr.jsonl', 'w', encoding='utf-8') as file:
    data_train_true.to_json(file, orient="records", lines=True, force_ascii=False)